In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, time, math, json, sqlite3, requests
from itertools import product
from typing import List, Tuple

API_URL   = "https://www.googleapis.com/books/v1/volumes"
API_KEY   = None  # ضيفي مفتاحك هون لو عندك (مثلاً "AIza...") أو خليه None
PER_PAGE  = 40
PER_QUERY_CAP = 1000
SLEEP_BASE = 0.25

# 📌 مسار قاعدة البيانات (يتسيف بالفولدر المطلوب)
DB_PATH = r"C:\Users\Rama Al Jada\Desktop\testbook\books_crawl.sqlite"
TABLE_BOOKS = "books"
TABLE_STATE = "crawl_state"

# --------- 1) توليد الباكتات ----------
letters = [chr(c) for c in range(ord('a'), ord('z')+1)]
digits  = [str(d) for d in range(10)]
author_prefixes = letters + digits   # 36 بادئة
title_prefixes  = letters            # 26 بادئة

def generate_buckets() -> List[Tuple[str, str, str]]:
    """
    كل باكت عبارة عن (q_string, bucket_key, hint)
    """
    buckets = []

    # الباكتات الأساسية (حرف واحد للعنوان × حرف/رقم للمؤلف)
    for t, a in product(title_prefixes, author_prefixes):
        q = f'intitle:{t} inauthor:{a}'
        key = f'intitle_{t}__inauthor_{a}'
        buckets.append((q, key, f"title starts {t}, author starts {a}"))

    # ✅ ثنائيات العنوان (aa, ab, ..., zz)
    for t2 in (f"{x}{y}" for x,y in product(letters, letters)):
        q = f'intitle:{t2}'
        key = f'intitle_{t2}'
        buckets.append((q, key, f"title starts {t2}"))

    return buckets

# --------- 2) قاعدة البيانات ----------
def init_db():
    con = sqlite3.connect(DB_PATH)
    cur = con.cursor()
    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {TABLE_BOOKS} (
        id TEXT PRIMARY KEY,
        title TEXT,
        authors TEXT,
        publishedDate TEXT,
        industryIds TEXT,
        raw JSON
    );
    """)
    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {TABLE_STATE} (
        bucket_key TEXT PRIMARY KEY,
        q TEXT NOT NULL,
        next_start INTEGER NOT NULL DEFAULT 0,
        done INTEGER NOT NULL DEFAULT 0,
        total_reported INTEGER
    );
    """)
    con.commit()
    con.close()

def save_state(bucket_key, q, next_start, done, total_reported=None):
    con = sqlite3.connect(DB_PATH)
    cur = con.cursor()
    cur.execute(f"""
    INSERT INTO {TABLE_STATE} (bucket_key, q, next_start, done, total_reported)
    VALUES (?, ?, ?, ?, ?)
    ON CONFLICT(bucket_key)
    DO UPDATE SET next_start=excluded.next_start, done=excluded.done,
                  total_reported=COALESCE(excluded.total_reported, {TABLE_STATE}.total_reported);
    """, (bucket_key, q, next_start, done, total_reported))
    con.commit()
    con.close()

def load_state(bucket_key):
    con = sqlite3.connect(DB_PATH)
    cur = con.cursor()
    cur.execute(f"SELECT q, next_start, done, total_reported FROM {TABLE_STATE} WHERE bucket_key=?;", (bucket_key,))
    row = cur.fetchone()
    con.close()
    if row:
        return {"q": row[0], "next_start": row[1], "done": row[2], "total_reported": row[3]}
    return None

def insert_books(items):
    if not items:
        return 0
    con = sqlite3.connect(DB_PATH)
    cur = con.cursor()
    count = 0
    for it in items:
        bid = it.get("id")
        if not bid:
            continue
        vol = it.get("volumeInfo", {})
        title = vol.get("title")
        authors = ", ".join(vol.get("authors", [])) if vol.get("authors") else None
        pubdate = vol.get("publishedDate")
        inds = vol.get("industryIdentifiers", [])
        inds_str = json.dumps(inds, ensure_ascii=False)
        raw = json.dumps(it, ensure_ascii=False)
        try:
            cur.execute(f"""
            INSERT OR IGNORE INTO {TABLE_BOOKS}(id, title, authors, publishedDate, industryIds, raw)
            VALUES (?, ?, ?, ?, ?, ?);
            """, (bid, title, authors, pubdate, inds_str, raw))
            count += cur.rowcount
        except sqlite3.Error:
            pass
    con.commit()
    con.close()
    return count

# --------- 3) طلب API مع Backoff ----------
def get_page(q, start_index=0, per_page=PER_PAGE):
    params = {"q": q, "maxResults": per_page, "startIndex": start_index}
    if API_KEY:
        params["key"] = API_KEY

    backoff = SLEEP_BASE
    for attempt in range(6):
        r = requests.get(API_URL, params=params, timeout=30)
        if r.status_code == 200:
            return r.json()
        elif r.status_code in (429, 403, 500, 503):
            time.sleep(backoff)
            backoff = min(backoff * 2, 8.0)
        else:
            raise requests.HTTPError(f"{r.status_code} {r.reason} — {r.url}")
    r.raise_for_status()

# --------- 4) زحف باكت واحد ----------
def crawl_bucket(q, bucket_key):
    st = load_state(bucket_key)
    start = 0
    total_reported = None

    if st:
        if st["done"]:
            return 0, 0, st.get("total_reported")
        start = st["next_start"] or 0
        total_reported = st.get("total_reported")

    if start == 0:
        first = get_page(q, start_index=0)
        total_reported = first.get("totalItems", 0)
        items = first.get("items", []) or []
        insert_books(items)
        start = len(items)
        save_state(bucket_key, q, start, 0, total_reported)
        time.sleep(SLEEP_BASE)

    inserted_total = 0
    while start < min(total_reported or PER_QUERY_CAP, PER_QUERY_CAP):
        try:
            data = get_page(q, start_index=start)
        except requests.HTTPError as e:
            print(f"[{bucket_key}] HTTPError at start={start}: {e}")
            break
        items = data.get("items", []) or []
        if not items:
            break
        inserted = insert_books(items)
        inserted_total += inserted
        start += len(items)
        save_state(bucket_key, q, start, 0, total_reported)
        time.sleep(SLEEP_BASE)

    save_state(bucket_key, q, start, 1, total_reported)
    return inserted_total, start, total_reported

# --------- 5) زحف الكل ----------
def crawl_all(target_count=1_000_000):
    init_db()
    buckets = generate_buckets()
    total_before = count_books()
    print(f"Existing in DB before: {total_before}")

    for i, (q, key, hint) in enumerate(buckets, 1):
        st = load_state(key)
        if st and st["done"]:
            continue
        print(f"[{i}/{len(buckets)}] Bucket: {key} — {hint}")
        inserted, fetched, total_reported = crawl_bucket(q, key)
        current = count_books()
        print(f"  inserted:{inserted}  fetched≈{fetched}  total_reported={total_reported}  DB now:{current}")
        if current >= target_count:
            print("Reached target. Stopping.")
            break

    print(f"Final DB count: {count_books()}")

def count_books():
    con = sqlite3.connect(DB_PATH)
    cur = con.cursor()
    cur.execute(f"SELECT COUNT(*) FROM {TABLE_BOOKS};")
    c = cur.fetchone()[0]
    con.close()
    return c

if __name__ == "__main__":
    crawl_all(target_count=1_000_000)

In [ ]:
import sqlite3
import pandas as pd

DB_PATH = r"C:\Users\Rama Al Jada\Desktop\testbook\books_crawl.sqlite"
TABLE_BOOKS = "books"

# اتصال بالقاعدة
con = sqlite3.connect(DB_PATH)

# جلب أول صف فقط لمعرفة الأعمدة
df = pd.read_sql_query(f"SELECT * FROM {TABLE_BOOKS} LIMIT 1", con)
con.close()

# عرض أسماء الأعمدة وعددها
print("Columns:", df.columns.tolist())
print("Number of columns:", len(df.columns))

In [ ]:
import sqlite3
import pandas as pd

DB_PATH = r"C:\Users\Rama Al Jada\Desktop\testbook\books_crawl.sqlite"
TABLE_BOOKS = "books"

# اتصال بالقاعدة
con = sqlite3.connect(DB_PATH)

# قراءة الجدول
df = pd.read_sql_query(f"SELECT * FROM {TABLE_BOOKS}", con)
con.close()

# عدد الصفوف (الريكورد)
total_records = len(df)
print(f"Total records in DB: {total_records}\n")

# حساب عدد القيم الفارغة لكل عمود
null_counts = df.isnull().sum()

# عرض النتيجة
for col, nulls in null_counts.items():
    print(f"{col}: {nulls} null values")

In [ ]:
import sqlite3
import pandas as pd

DB_PATH = r"C:\Users\Rama Al Jada\Desktop\testbook\books_crawl.sqlite"
TABLE_BOOKS = "books"

# اتصال بالقاعدة
con = sqlite3.connect(DB_PATH)

# قراءة الجدول
df = pd.read_sql_query(f"SELECT * FROM {TABLE_BOOKS}", con)
con.close()

# طباعة أول 5 صفوف لكل عمود
for col in df.columns:
    print(f"--- Column: {col} ---")
    print(df[col].head(5))
    print("\n")

In [ ]:
import shutil

db_path = r"C:\Users\Rama Al Jada\Desktop\testbook\books_crawl.sqlite"
new_db = r"C:\Users\Rama Al Jada\Desktop\testbook\new_books_crawl.sqlite"

shutil.copyfile(db_path, new_db)
print("Copy created successfully!")

In [ ]:
import sqlite3
import pandas as pd

# مسار الداتا بيس الجديدة
new_db = r"C:\Users\Rama Al Jada\Desktop\testbook\new_books_crawl.sqlite"
TABLE_BOOKS = "books"

# فتح الاتصال بالداتا بيس
con = sqlite3.connect(new_db)

# قراءة الجدول
try:
    df = pd.read_sql_query(f"SELECT * FROM {TABLE_BOOKS}", con)
except Exception as e:
    print(f"Error: {e}")
    con.close()
    raise

con.close()

# عدد الصفوف (الريكورد)
total_records = len(df)
print(f"Total records in DB: {total_records}\n")

# حساب عدد القيم الفارغة لكل عمود
null_counts = df.isnull().sum()

# عرض النتيجة
for col, nulls in null_counts.items():
    print(f"{col}: {nulls} null values")

In [ ]:
import sqlite3
import pandas as pd
import json

# مسار الداتا بيس الجديدة
new_db = r"C:\Users\Rama Al Jada\Desktop\testbook\new_books_crawl.sqlite"
TABLE_BOOKS = "books"

# فتح الاتصال
con = sqlite3.connect(new_db)
df = pd.read_sql_query(f"SELECT * FROM {TABLE_BOOKS}", con)

In [ ]:
# نعمل أعمدة جديدة من industryIds
def extract_industry_ids(industry_json):
    try:
        data = json.loads(industry_json)
        # نجمع كل identifiers بفاصلة
        return ', '.join(item.get("identifier", "") for item in data)
    except:
        return None

df['industry_identifiers'] = df['industryIds'].apply(extract_industry_ids)

In [ ]:
def extract_features(raw_json):
    try:
        data = json.loads(raw_json)
        volume_info = data.get("volumeInfo", {})
        sale_info = data.get("saleInfo", {})
        access_info = data.get("accessInfo", {})

        return pd.Series({
            "selfLink": data.get("selfLink"),
            "title": volume_info.get("title"),
            "authors": ', '.join(volume_info.get("authors", [])) if volume_info.get("authors") else None,
            "publishedDate": volume_info.get("publishedDate"),
            "description": volume_info.get("description"),
            "reading_text": volume_info.get("readingModes", {}).get("text"),
            "reading_image": volume_info.get("readingModes", {}).get("image"),
            "pageCount": volume_info.get("pageCount"),
            "printType": volume_info.get("printType"),
            "categories": ', '.join(volume_info.get("categories", [])) if volume_info.get("categories") else None,
            "thumbnail": volume_info.get("imageLinks", {}).get("thumbnail"),
            "language": volume_info.get("language"),
            "infoLink": volume_info.get("infoLink"),
            "saleability": sale_info.get("saleability"),
            "isEbook": sale_info.get("isEbook"),
            "pdf_available": access_info.get("pdf", {}).get("isAvailable")
        })
    
    except:
        return pd.Series([None]*17, index=[
            "selfLink","title","authors","publishedDate","description",
            "reading_text","reading_image","pageCount","printType",
            "categories","thumbnail","language","infoLink","saleability",
            "isEbook","pdf_available"
        ])

# نفك كل الفيتشرات من raw
features_df = df['raw'].apply(extract_features)

In [ ]:
df_final = pd.concat([df, features_df], axis=1)

In [ ]:
print(df_final.columns[df_final.columns.duplicated()])

In [ ]:
df_final = df_final.loc[:, ~df_final.columns.duplicated()]

In [ ]:
print(df_final.columns)

In [ ]:
con = sqlite3.connect(new_db)
df_final.to_sql(TABLE_BOOKS, con, if_exists="replace", index=False)
con.close()
print("Features extracted and saved successfully!")

In [ ]:
import sqlite3
import pandas as pd

con = sqlite3.connect(new_db)
df_check = pd.read_sql_query(f"SELECT * FROM {TABLE_BOOKS}", con)
con.close()

print(f"Total records in new DB: {len(df_check)}")

In [ ]:
print(df_check.columns)

In [ ]:
print(df_check.head())

In [ ]:
# عدد الأعمدة في الداتا فريم
num_columns = len(df_final.columns)
print(f"Number of columns: {num_columns}")

In [ ]:
# حساب عدد القيم الفارغة لكل عمود
null_counts = df_final.isnull().sum()

# عرض النتيجة
for col, nulls in null_counts.items():
    print(f"{col}: {nulls} null values")

In [ ]:
import sqlite3

# نحذف العمودين من الداتا فريم
df_final = df_final.drop(columns=['raw', 'industryIds'])

# نكتب الداتا فريم المحدثة بالداتا بيس
con = sqlite3.connect(new_db)  # الداتا بيس الجديد
df_final.to_sql(TABLE_BOOKS, con, if_exists="replace", index=False)
con.close()

print("raw و industryIds تم حذفهم والداتا بيس تم تحديثها بنجاح!")

In [ ]:
# عدد الأعمدة في الداتا فريم
num_columns = len(df_final.columns)
print(f"Number of columns: {num_columns}")


In [ ]:
# حساب عدد القيم الفارغة لكل عمود
null_counts = df_final.isnull().sum()

# عرض النتيجة
for col, nulls in null_counts.items():
    print(f"{col}: {nulls} null values")

In [ ]:
# فتح الداتا بيس الجديد
import sqlite3
import pandas as pd

con = sqlite3.connect(new_db)
df_sample = pd.read_sql_query(f"SELECT * FROM {TABLE_BOOKS} LIMIT 5", con)
con.close()

# عرض العينة
print(df_sample)

In [ ]:
# عرض جميع التصنيفات الفريدة وعدد الكتب بكل تصنيف
category_counts = df_final['categories'].value_counts(dropna=False)
print(category_counts)


# cleaning

In [ ]:
import sqlite3
import pandas as pd

# مسار قاعدة البيانات
new_db = r"C:\Users\Rama Al Jada\Desktop\testbook\new_books_crawl.sqlite"
TABLE_BOOKS = "books"

# الاتصال
con = sqlite3.connect(new_db)

# قراءة البيانات
df = pd.read_sql_query(f"SELECT * FROM {TABLE_BOOKS}", con)

print("قبل الحذف:", len(df))

# حذف الصفوف اللي فيها null بـ title أو authors
df = df.dropna(subset=["title", "authors"])

print("بعد الحذف:", len(df))

# حفظ التغييرات في الداتا بيس
df.to_sql(TABLE_BOOKS, con, if_exists="replace", index=False)
con.close()

print("تم حذف الصفوف اللي فيها null بـ title أو authors وحفظ التغييرات.")

In [ ]:
import sqlite3
import pandas as pd

# الاتصال بالداتا بيس
new_db = r"C:\Users\Rama Al Jada\Desktop\testbook\new_books_crawl.sqlite"
TABLE_BOOKS = "books"

con = sqlite3.connect(new_db)

# قراءة الداتا
df = pd.read_sql_query(f"SELECT * FROM {TABLE_BOOKS}", con)

# حذف الصفوف اللي فيها null بالـ title أو authors
df = df.dropna(subset=["title", "authors"])

# نشوف nulls بعد التنظيف
null_counts = df.isnull().sum()

print("عدد القيم الفارغة بعد التنظيف:\n")
for col, nulls in null_counts.items():
    print(f"{col}: {nulls} null values")

# نرجع نخزن الداتا المنظفة مكان الجدول
df.to_sql(TABLE_BOOKS, con, if_exists="replace", index=False)

con.close()
print("\nتم الحذف والتخزين بنجاح ✅")